# Tools on LangChain

In [ ]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-2.0-flash-001")
question = "how old is the US president?"

Modern LLMs hide the need to construct a prompt from the user, you can define your tools as schemas instead and pass them as a separate argument:

In [ ]:
search_tool = {
    "description": "Returns about common facts, fresh events and news from Google Search engine based on a query.",
    "title": "google_search",
    "type": "object",
    "properties": {
        "query": {
            "description": "Search query to be sent to the search engine",
            "title": "search_query",
            "type": "string"},
    },
    "required": ["query"]
}
step1 = llm.invoke(question, tools=[search_tool])

As we can see, now our outputs contains a special part called `tool_calls` - again, there's no need for us to parse the output any more, it's all done by an LLM:

In [ ]:
print(step1.tool_calls)

[{'name': 'google_search', 'args': {'query': 'age of Donald Trump'}, 'id': 'd4dc04b5-a554-410a-9734-765e44cefa91', 'type': 'tool_call'}]


We can pass the tool calling result back to an LLM as a special `ToolMessage` message:

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

tool_result = ToolMessage(content="Donald Trump › Age 78 years June 14, 1946\n", tool_call_id=step1.tool_calls[0]["id"])
step2 = llm.invoke([
    HumanMessage(content=question), step1, tool_result], tools=[search_tool])
assert len(step2.tool_calls) == 0

print(step2.content)

Donald Trump is 78 years old.



For the convinience, we can also `bind` tools to an LLM so that they would be auto-added to arguments on every invocation:

In [ ]:
llm_with_tools = llm.bind(tools=[search_tool])
llm_with_tools.invoke(question)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'google_search', 'arguments': '{"query": "age of US president Joe Biden"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 36, 'candidates_token_count': 10, 'total_token_count': 46, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.07167894244194031}, id='run-519331b6-82a5-4f9b-9102-2b123544346b-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'age of US president Joe Biden'}, 'id': '12dbb680-e7fe-4c06-9f54-cfcb68070c61', 'type': 'tool_call'}], usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})